In [2]:
import pandas as pd
import numpy as np
import json, urllib
from datetime import datetime
import matplotlib.pyplot as plt
import math

#### Sheets Consolidated

In [3]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/MoveInSyncData-consolidated.csv'
df = pd.read_csv(datalocation)
original_data = df

In [4]:
df = original_data

#### Split Lat and Long

In [5]:
df[['pickup_geoX','pickup_geoY']] = df['pickup_geo'].str.split(',',expand=True)
df[['drop_geoX','drop_geoY']] = df['drop_geo'].str.split(',',expand=True)

In [6]:
df['pickup_geoX'] = df['pickup_geoX'].astype(float)
df['pickup_geoY'] = df['pickup_geoY'].astype(float)
df['drop_geoX'] = df['drop_geoX'].astype(float)
df['drop_geoY'] = df['drop_geoY'].astype(float)

#### Protect Privacy of Company

In [7]:
df['Company'] = 0
df.loc[df['bunit_id']=='att-ATT','Company'] = 1
df.loc[df['bunit_id']=='dxcshell-DBlr','Company'] = 2
df.loc[df['bunit_id']=='cerner-GTP','Company'] = 3
df.loc[df['bunit_id']=='cerner-MBlr','Company'] = 4
df.loc[df['bunit_id']=='fico-FBlr','Company'] = 5
df.loc[df['bunit_id']=='hbc-HBlr','Company'] = 6
df.loc[df['bunit_id']=='ensono-Ensono','Company'] = 7
df.loc[df['bunit_id']=='nutanix-NBlr','Company'] = 8
df.loc[df['bunit_id']=='infosys-IBlr','Company'] = 9
df.loc[df['bunit_id']=='rideinsync-blr','Company'] = 10

#### Distance Calculation

In [8]:
lon2 = df['drop_geoY']
lat2 = df['drop_geoX']
lon1 = (df['pickup_geoY'])
lat1 = (df['pickup_geoX'])

radius = 6371  # km
dlat = (lat2 - lat1)
dlon = (lon2 - lon1)
dlat = [math.sin(math.radians(x)/2)*math.sin(math.radians(x)/2) for x in dlat]
dlon = [math.sin(math.radians(x)/2)*math.sin(math.radians(x)/2) for x in dlon]
cosradlat1 = [math.cos(math.radians(x)) for x in lat1]
cosradlat2 = [math.cos(math.radians(x)) for x in lat2]
a = dlat + np.multiply(np.multiply(cosradlat1,cosradlat2),dlon)
c = [2 * math.atan2(math.sqrt(x), math.sqrt(1 - x)) for x in a]
df['distance'] = [radius * x for x in c]

#### Extracting Time, Month, Hour, Minute, Day of Week from Planned Pickup Time and Shift Time

In [9]:
df.planned_pickup_time.head()

0    2019-05-16 06:45:00
1    2019-05-16 06:26:00
2    2019-05-16 15:45:00
3    2019-05-16 01:45:00
4    2019-05-16 06:09:00
Name: planned_pickup_time, dtype: object

In [10]:
x = list(df.planned_pickup_time)
df['dayofweek'] = [str(p)[5:10] for p in x]
df['Hourpickup']= [str(p)[11:13] for p in x]
df['Minutepickup']= [str(p)[14:16] for p in x]
df['month']= [str(p)[5:7] for p in x]

In [11]:
x = list(df.shift_time_type)
df['hour_shift']= [str(p)[:2] for p in x]
df['minute_shift']= [str(p)[3:5] for p in x]

In [12]:
storedf = df

In [13]:
original_data.columns

Index(['shift_time_type', 'date', 'bunit_id', 'profile_office', 'employee_id',
       'trip_id', 'trip_type', 'actual_cab_registration',
       'actual_vendor_cab_id', 'vendor_id', 'gender', 'planned_escort',
       'planned_km', 'traveled_km', 'employee_order', 'employee_count',
       'required_escort', 'cab_type', 'trip_state_text', 'pickup_geo',
       'drop_geo', 'planned_pickup_time', 'pickup_geoX', 'pickup_geoY',
       'drop_geoX', 'drop_geoY', 'Company', 'distance', 'dayofweek',
       'Hourpickup', 'Minutepickup', 'month', 'hour_shift', 'minute_shift'],
      dtype='object')

#### Extracting Relevant Features

In [17]:
features_req = ['employee_id','trip_id','trip_type','gender','cab_type', 'trip_state_text','planned_pickup_time', 'pickup_geoX', 'pickup_geoY',
       'drop_geoX', 'drop_geoY', 'distance', 'Company', 'dayofweek', 'Hourpickup', 'Minutepickup',
       'month']

In [18]:
cleaneddf = df[features_req]

#### Removing Data

In [19]:
df1 = cleaneddf

In [20]:
df1.shape

(896047, 17)

#### Extracting only Completed Trips

In [21]:
df1 = df1[df1.trip_state_text == 'COMPLETED']
df1.shape

(700506, 17)

#### Removing Chennai and Pune Data

In [22]:
df1 = df1[(df1['pickup_geoX'] < 17) | (df1['pickup_geoX'] == 0 ) | (df1['pickup_geoX'].isnull())]
df1 = df1[(df1['pickup_geoY'] < 79) | (df1['pickup_geoY'] == 0 ) | (df1['pickup_geoY'].isnull())]
df1 = df1[(df1['drop_geoY'] < 80) | (df1['drop_geoY'] == 0 ) | (df1['drop_geoY'].isnull())]
df1 = df1[(df1['drop_geoX'] < 13.4) | (df1['drop_geoX'] == 0 ) | (df1['drop_geoX'].isnull())]
df1.shape

(679815, 17)

#### Removing rows with no Employee ID ( same as the rows with no Gender)

In [24]:
df1 = df1[~df1.employee_id.isnull()]
df1.shape

(677787, 17)

In [25]:
df1.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text', 'planned_pickup_time', 'pickup_geoX', 'pickup_geoY',
       'drop_geoX', 'drop_geoY', 'distance', 'Company', 'dayofweek',
       'Hourpickup', 'Minutepickup', 'month'],
      dtype='object')

In [26]:
df1.describe()

,trip_id,pickup_geoX,pickup_geoY,drop_geoX,drop_geoY,distance,Company
count,6.777870e+05,647988.000000,647988.000000,647988.000000,647988.000000,647988.000000,677787.000000
mean,7.856079e+05,12.880570,77.404153,12.895569,77.444947,20.083719,7.178574
std,3.964730e+05,0.722697,4.317989,0.650422,3.881074,280.994121,2.915860
min,2.852000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4.688025e+05,12.850023,77.621961,12.851022,77.613263,5.516659,6.000000
50%,1.029687e+06,12.908444,77.663161,12.920840,77.656903,10.755823,9.000000
75%,1.066434e+06,12.984913,77.667328,12.984987,77.670348,16.222257,9.000000
max,1.122656e+06,13.214574,77.844961,13.214552,77.844875,8685.541487,10.000000


#### Imputing the lat long missing data after sending data to excel. Importing back to python

In [27]:
nolatlong = df1[(df1.distance >= 40) | (df1.distance== 0) | (df1.distance.isnull())]
features = ['employee_id','trip_id','trip_type','pickup_geoX', 'pickup_geoY','drop_geoX', 'drop_geoY']
find = nolatlong[features]
yeslatlong = df1[~((df1.distance >= 40) | (df1.distance== 0) | (df1.distance.isnull()))]
data = yeslatlong[features]

In [28]:
find.to_csv('nolatlong.csv', encoding='utf-8', index=False)
data.to_csv('original.csv', encoding='utf-8', index=False)

In [29]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/missinglatlong.csv'
dfmiss = pd.read_csv(datalocation)

In [30]:
dfmiss.head()

,hey,employee_id,trip_id,pickup_geoX,pickup_geoY,drop_geoX,drop_geoY
0,ATT-0012461458,ATT-0012,461458,13.000378,77.709059,13.000918,77.631571
1,ATT-0022461946,ATT-0022,461946,13.008054,77.716602,13.008505,77.674678
2,ATT-0023462022,ATT-0023,462022,12.924350,77.671087,12.927514,77.672085
3,ATT-0086461496,ATT-0086,461496,12.938461,77.671299,12.938233,77.730927
4,ATT-0108462002,ATT-0108,462002,12.959024,77.708128,12.786387,77.768996


In [31]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/MoveInSyncData-cleaned.csv'
df2 = pd.read_csv(datalocation)

#### Removing the rows with no lat long location even after imputation

In [32]:
df3 = df2[~((df2.pickup_geoX == 0) | (df2.pickup_geoY == 0) | (df2.drop_geoX == 0) | (df2.drop_geoY == 0)) ]

In [33]:
df3.shape

(675761, 23)

In [34]:
df3.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text', 'planned_pickup_time', 'pickup_geoX', 'pickup_geoY',
       'drop_geoX', 'drop_geoY', 'distance', 'employee_count_y',
       'required_escort_y', 'max_distance', 'total_distance', 'costoftrip',
       'perpersoncost', 'Company', 'dayofweek', 'Hourpickup', 'Minutepickup',
       'month'],
      dtype='object')

#### Calculating the distance again for new found lat longs

In [35]:
lon2 = df3['drop_geoY']
lat2 = df3['drop_geoX']
lon1 = (df3['pickup_geoY'])
lat1 = (df3['pickup_geoX'])

radius = 6371  # km
dlat = (lat2 - lat1)
dlon = (lon2 - lon1)
dlat = [math.sin(math.radians(x)/2)*math.sin(math.radians(x)/2) for x in dlat]
dlon = [math.sin(math.radians(x)/2)*math.sin(math.radians(x)/2) for x in dlon]
cosradlat1 = [math.cos(math.radians(x)) for x in lat1]
cosradlat2 = [math.cos(math.radians(x)) for x in lat2]
a = dlat + np.multiply(np.multiply(cosradlat1,cosradlat2),dlon)
c = [2 * math.atan2(math.sqrt(x), math.sqrt(1 - x)) for x in a]
df3['distance'] = [radius * x for x in c]

C:\Users\yours\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
store2 = df3

In [72]:
dropping = ['employee_count_y','required_escort_y', 'max_distance', 'total_distance', 'costoftrip','perpersoncost']
df3 = df3.drop(columns=dropping)

In [73]:
df3.shape

(675761, 17)

#### Removing duplicates

In [74]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/keys.csv'
merging = pd.read_csv(datalocation)

In [75]:
merging.shape

(698478, 5)

In [76]:
newmerging = merging.drop_duplicates()
newmerging.shape

(623415, 5)

In [77]:
final = pd.merge(df3,newmerging, on=['employee_id','trip_id'], how = 'left')

In [78]:
df3.shape

(675761, 17)

In [79]:
df4 = df3.drop_duplicates()
df4.shape

(600841, 17)

In [80]:
df4 = final.drop_duplicates()

#### Cost Attribution

In [81]:
attr = df4.groupby(['trip_id']).agg(
    {
        'employee_count':max,
        'required_escort':sum,
        'distance':max,
    }
)
attr2 = df4.groupby(['trip_id']).agg(
    {
        'distance':sum,
    }
)

In [82]:
attr = pd.merge(attr,attr2, on=['trip_id'], how = 'left')

In [83]:
attr.rename(columns={"distance_x": "max_distance"}, inplace = True)
attr.rename(columns={"distance_y": "total_distance"}, inplace = True) 

In [84]:
attr.loc[attr['required_escort']>0,'required_escort'] = 1

In [85]:
attr.head()

,employee_count,required_escort,max_distance,total_distance
trip_id,,,,
2852,1,0.0,8.531675,8.531675
2853,2,1.0,15.560621,16.360969
2854,2,0.0,10.352357,17.590432
2855,1,0.0,18.032145,18.032145
2856,1,0.0,9.004197,9.004197


In [86]:
attr['costoftrip'] = (attr['max_distance'] + attr['max_distance']*attr['required_escort'])*10

In [87]:
attr.head()

,employee_count,required_escort,max_distance,total_distance,costoftrip
trip_id,,,,,
2852,1,0.0,8.531675,8.531675,85.316749
2853,2,1.0,15.560621,16.360969,311.212419
2854,2,0.0,10.352357,17.590432,103.523574
2855,1,0.0,18.032145,18.032145,180.321446
2856,1,0.0,9.004197,9.004197,90.041974


In [88]:
x = pd.merge(df4,attr, on=['trip_id'], how = 'left')

In [89]:
df4 = x

In [90]:
df4.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'Hourpickup', 'Minutepickup', 'month', 'employee_count_x',
       'required_escort_x', 'trip_state_text_y', 'employee_count_y',
       'required_escort_y', 'max_distance', 'total_distance', 'costoftrip'],
      dtype='object')

#### Removing employee count = 0 rows (1800)

In [91]:
sum(df4.employee_count_y == 0)

1761

In [92]:
df5 = df4[~(df4.employee_count_x == 0)]

In [93]:
df5.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'Hourpickup', 'Minutepickup', 'month', 'employee_count_x',
       'required_escort_x', 'trip_state_text_y', 'employee_count_y',
       'required_escort_y', 'max_distance', 'total_distance', 'costoftrip'],
      dtype='object')

In [94]:
df5['perpersoncost'] = ((df5['distance']/df5['total_distance'])*df5['costoftrip']/2)  + (df5['costoftrip']/(2*df5['employee_count_y']))

C:\Users\yours\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
df5.perpersoncost.describe()

count    599041.000000
mean         58.252701
std          58.584181
min           0.000556
25%          20.348198
50%          43.153307
75%          74.727419
max         547.073084
Name: perpersoncost, dtype: float64

#### Imputing missing Planned Pick up time column

In [97]:
sum(df5.planned_pickup_time.isnull())

110207

In [99]:
df5.shape

(599041, 26)

In [98]:
df5.head()

,employee_id,trip_id,trip_type,gender,cab_type,trip_state_text_x,planned_pickup_time,pickup_geoX,pickup_geoY,drop_geoX,...,month,employee_count_x,required_escort_x,trip_state_text_y,employee_count_y,required_escort_y,max_distance,total_distance,costoftrip,perpersoncost
0,ATT-0001,461353,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 6:45,12.923583,77.671061,12.988366,...,5.0,2,False,COMPLETED,2,0.0,15.506603,30.629322,155.066034,77.047147
1,ATT-0002,461528,LOGIN,MALE,4 Seater,COMPLETED,5/16/2019 6:26,12.955146,77.738991,12.934748,...,5.0,1,False,COMPLETED,1,0.0,5.301257,7.177641,53.012574,46.083283
2,ATT-0003,461849,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 15:45,12.924626,77.671098,12.968512,...,5.0,3,False,COMPLETED,3,0.0,16.542731,46.928047,165.427313,56.728831
3,ATT-0004,461431,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 1:45,12.924514,77.671051,13.043238,...,5.0,3,True,COMPLETED,3,1.0,22.936604,59.086995,458.732089,128.031994
4,ATT-0005,461915,LOGIN,FEMALE,4 Seater,COMPLETED,5/16/2019 6:09,12.921150,77.646636,12.923322,...,5.0,2,True,COMPLETED,2,1.0,2.596742,5.193483,51.934835,25.967417


In [100]:
x = list(original_data.shift_time_type)
original_data['hour_shift']= [str(p)[:2] for p in x]
original_data['minute_shift']= [str(p)[3:5] for p in x]

In [101]:
original_data.columns

Index(['shift_time_type', 'date', 'bunit_id', 'profile_office', 'employee_id',
       'trip_id', 'trip_type', 'actual_cab_registration',
       'actual_vendor_cab_id', 'vendor_id', 'gender', 'planned_escort',
       'planned_km', 'traveled_km', 'employee_order', 'employee_count',
       'required_escort', 'cab_type', 'trip_state_text', 'pickup_geo',
       'drop_geo', 'planned_pickup_time', 'pickup_geoX', 'pickup_geoY',
       'drop_geoX', 'drop_geoY', 'Company', 'distance', 'dayofweek',
       'Hourpickup', 'Minutepickup', 'month', 'hour_shift', 'minute_shift'],
      dtype='object')

In [102]:
features = ['trip_id','employee_id','hour_shift','minute_shift']

In [103]:
shiftmerge = original_data[features]

In [104]:
dummy = pd.merge(df5,shiftmerge, on=['employee_id','trip_id'], how = 'left')

In [105]:
df5.shape

(599041, 26)

In [106]:
dummy1 =dummy.drop_duplicates()

In [107]:
dummy1.shape

(599041, 28)

In [109]:
df5 = dummy1

In [110]:
df5.dtypes

employee_id             object
trip_id                  int64
trip_type               object
gender                  object
cab_type                object
trip_state_text_x       object
planned_pickup_time     object
pickup_geoX            float64
pickup_geoY            float64
drop_geoX              float64
drop_geoY              float64
distance               float64
Company                  int64
dayofweek               object
Hourpickup             float64
Minutepickup           float64
month                  float64
employee_count_x         int64
required_escort_x         bool
trip_state_text_y       object
employee_count_y         int64
required_escort_y      float64
max_distance           float64
total_distance         float64
costoftrip             float64
perpersoncost          float64
hour_shift              object
minute_shift            object
dtype: object

In [124]:
sum(df5.Hourpickup.isnull())

110207

In [114]:
df5.hour_shift.unique()

array(['06', '15', '01', '07', '17', '22', '21', '12', '16', '14', '05',
       '13', '02', '23', '03', '00', '10', '18', '04', 'Ad', 'No', '20',
       '19', '11', '09', '08'], dtype=object)

In [115]:
features = ['trip_id','employee_id','trip_type','distance','Hourpickup','Minutepickup','hour_shift','minute_shift']

In [116]:
plannedtime = df5[features]

In [120]:
plannedtime = plannedtime[~((plannedtime.hour_shift == 'Ad') | (plannedtime.hour_shift == 'No'))]

In [121]:
plannedtime.to_csv('plannedtime.csv', encoding='utf-8', index=False)

In [131]:
df5[~(((df5.hour_shift == 'Ad') | (df5.hour_shift == 'No')) & (df5.Hourpickup.isnull()))].shape

(511140, 28)

In [132]:
df5.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'Hourpickup', 'Minutepickup', 'month', 'employee_count_x',
       'required_escort_x', 'trip_state_text_y', 'employee_count_y',
       'required_escort_y', 'max_distance', 'total_distance', 'costoftrip',
       'perpersoncost', 'hour_shift', 'minute_shift'],
      dtype='object')

In [133]:
features = ['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'month', 'employee_count_x',
       'required_escort_x', 'trip_state_text_y', 'employee_count_y',
       'required_escort_y', 'max_distance', 'total_distance', 'costoftrip',
       'perpersoncost', 'hour_shift', 'minute_shift']

In [134]:
df6 = df5[features]

In [135]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/plannedtimefinds.csv'
merging = pd.read_csv(datalocation)

In [136]:
merging.head()

,trip_id,employee_id,trip_type,distance,Hourpickup,Minutepickup,hour_shift,minute_shift,Delay
0,461353,ATT-0001,LOGOUT,15.122719,6,45,6,30,-15.0
1,461528,ATT-0002,LOGIN,5.301257,6,26,6,45,19.0
2,461849,ATT-0003,LOGOUT,16.542731,15,45,15,30,-15.0
3,461431,ATT-0004,LOGOUT,13.286662,1,45,1,30,-15.0
4,461915,ATT-0005,LOGIN,2.596742,6,9,6,15,6.0


In [137]:
features = ['trip_id','employee_id','Hourpickup','Minutepickup']
mergingf = merging[features]

In [139]:
final2 = pd.merge(df6,mergingf, on=['trip_id','employee_id'], how = 'left')

In [142]:
x = final2.drop_duplicates()
x.shape

(599041, 28)

In [151]:
y = final2[~final2.Hourpickup.isnull()]

In [153]:
df7 = y

In [167]:
f = ['trip_id','employee_id','month']
z = original_data[f]

In [170]:
df7.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'month', 'employee_count_x', 'required_escort_x',
       'trip_state_text_y', 'employee_count_y', 'required_escort_y',
       'max_distance', 'total_distance', 'costoftrip', 'perpersoncost',
       'hour_shift', 'minute_shift', 'Hourpickup', 'Minutepickup'],
      dtype='object')

In [172]:
f = ['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'employee_count_x', 'required_escort_x',
       'trip_state_text_y', 'employee_count_y', 'required_escort_y',
       'max_distance', 'total_distance', 'costoftrip', 'perpersoncost',
       'hour_shift', 'minute_shift', 'Hourpickup', 'Minutepickup']

In [173]:
df8 = df7[f]

In [174]:
res = pd.merge(df8,z, on=['trip_id','employee_id'], how = 'left')

In [178]:
bit = res.drop_duplicates()

In [181]:
df9 = bit

In [191]:
df9.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'employee_count_x', 'required_escort_x',
       'trip_state_text_y', 'employee_count_y', 'required_escort_y',
       'max_distance', 'total_distance', 'costoftrip', 'perpersoncost',
       'hour_shift', 'minute_shift', 'Hourpickup', 'Minutepickup', 'month'],
      dtype='object')

In [351]:
datalocation = 'C:/Users/yours/OneDrive/Documents/2019 - Cornell - Coursework/Big Messy/Project MoveInSync/Dayofweek.csv'
merging = pd.read_csv(datalocation)

#### Finding the day of week from Planned Pick up time. 

In [352]:
merging.head()

,dayofweek,daynum
0,05-01,4
1,05-02,5
2,05-03,6
3,05-04,7
4,05-05,1


In [199]:
df10 = df9

In [200]:
df9.columns

Index(['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'employee_count_x', 'required_escort_x',
       'trip_state_text_y', 'employee_count_y', 'required_escort_y',
       'max_distance', 'total_distance', 'costoftrip', 'perpersoncost',
       'hour_shift', 'minute_shift', 'Hourpickup', 'Minutepickup', 'month'],
      dtype='object')

In [280]:
yo = original_data.date
mon = [str(x)[5:7] for x in yo]
dmon = [str(x)[8:10] for x in yo]

In [243]:
f = ['trip_id','employee_id']
dummy = original_data[f]

In [282]:
dummy['month'] = mon
dummy['dateofmonth'] = dmon

C:\Users\yours\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\yours\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [283]:
f = ['employee_id', 'trip_id', 'trip_type', 'gender', 'cab_type',
       'trip_state_text_x', 'planned_pickup_time', 'pickup_geoX',
       'pickup_geoY', 'drop_geoX', 'drop_geoY', 'distance', 'Company',
       'dayofweek', 'employee_count_x', 'required_escort_x',
       'trip_state_text_y', 'employee_count_y', 'required_escort_y',
       'max_distance', 'total_distance', 'costoftrip', 'perpersoncost',
       'hour_shift', 'minute_shift', 'Hourpickup', 'Minutepickup']
df11 = df9[f]

In [284]:
res3 = pd.merge(df11,dummy, on=['trip_id','employee_id'], how = 'left')

In [333]:
x = res3.drop_duplicates()

In [334]:
df12 = x

In [339]:
df12.head()

,employee_id,trip_id,trip_type,gender,cab_type,trip_state_text_x,planned_pickup_time,pickup_geoX,pickup_geoY,drop_geoX,...,max_distance,total_distance,costoftrip,perpersoncost,hour_shift,minute_shift,Hourpickup,Minutepickup,month,dateofmonth
0,ATT-0001,461353,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 6:45,12.923583,77.671061,12.988366,...,15.506603,30.629322,155.066034,77.047147,06,30,6.0,45.0,05,16
1,ATT-0002,461528,LOGIN,MALE,4 Seater,COMPLETED,5/16/2019 6:26,12.955146,77.738991,12.934748,...,5.301257,7.177641,53.012574,46.083283,06,45,6.0,26.0,05,16
2,ATT-0003,461849,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 15:45,12.924626,77.671098,12.968512,...,16.542731,46.928047,165.427313,56.728831,15,30,15.0,45.0,05,16
3,ATT-0004,461431,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 1:45,12.924514,77.671051,13.043238,...,22.936604,59.086995,458.732089,128.031994,01,30,1.0,45.0,05,16
4,ATT-0005,461915,LOGIN,FEMALE,4 Seater,COMPLETED,5/16/2019 6:09,12.921150,77.646636,12.923322,...,2.596742,5.193483,51.934835,25.967417,06,15,6.0,9.0,05,16


In [340]:
df12['dayofweek'] = df12['month'] + '-' + df12['dateofmonth']

C:\Users\yours\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [353]:
res5 = pd.merge(df12,merging, on=['dayofweek'], how = 'left')

In [356]:
df13 = res5

In [358]:
df13.head()

,employee_id,trip_id,trip_type,gender,cab_type,trip_state_text_x,planned_pickup_time,pickup_geoX,pickup_geoY,drop_geoX,...,total_distance,costoftrip,perpersoncost,hour_shift,minute_shift,Hourpickup,Minutepickup,month,dateofmonth,daynum
0,ATT-0001,461353,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 6:45,12.923583,77.671061,12.988366,...,30.629322,155.066034,77.047147,06,30,6.0,45.0,05,16,5
1,ATT-0002,461528,LOGIN,MALE,4 Seater,COMPLETED,5/16/2019 6:26,12.955146,77.738991,12.934748,...,7.177641,53.012574,46.083283,06,45,6.0,26.0,05,16,5
2,ATT-0003,461849,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 15:45,12.924626,77.671098,12.968512,...,46.928047,165.427313,56.728831,15,30,15.0,45.0,05,16,5
3,ATT-0004,461431,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 1:45,12.924514,77.671051,13.043238,...,59.086995,458.732089,128.031994,01,30,1.0,45.0,05,16,5
4,ATT-0005,461915,LOGIN,FEMALE,4 Seater,COMPLETED,5/16/2019 6:09,12.921150,77.646636,12.923322,...,5.193483,51.934835,25.967417,06,15,6.0,9.0,05,16,5


#### Renaming Columns

In [359]:
df13.rename(columns={"dateofmonth": "date_of_month"}, inplace = True)
df13.rename(columns={"daynum": "day"}, inplace = True)
df13.rename(columns={"Minutepickup": "minute"}, inplace = True)
df13.rename(columns={"Hourpickup": "hour"}, inplace = True)

In [361]:
df13.rename(columns={"trip_state_text_x": "trip_state_text"}, inplace = True)
df13.head()

,employee_id,trip_id,trip_type,gender,cab_type,trip_state_text,planned_pickup_time,pickup_geoX,pickup_geoY,drop_geoX,...,total_distance,costoftrip,perpersoncost,hour_shift,minute_shift,hour,minute,month,date_of_month,day
0,ATT-0001,461353,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 6:45,12.923583,77.671061,12.988366,...,30.629322,155.066034,77.047147,06,30,6.0,45.0,05,16,5
1,ATT-0002,461528,LOGIN,MALE,4 Seater,COMPLETED,5/16/2019 6:26,12.955146,77.738991,12.934748,...,7.177641,53.012574,46.083283,06,45,6.0,26.0,05,16,5
2,ATT-0003,461849,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 15:45,12.924626,77.671098,12.968512,...,46.928047,165.427313,56.728831,15,30,15.0,45.0,05,16,5
3,ATT-0004,461431,LOGOUT,MALE,4 Seater,COMPLETED,5/16/2019 1:45,12.924514,77.671051,13.043238,...,59.086995,458.732089,128.031994,01,30,1.0,45.0,05,16,5
4,ATT-0005,461915,LOGIN,FEMALE,4 Seater,COMPLETED,5/16/2019 6:09,12.921150,77.646636,12.923322,...,5.193483,51.934835,25.967417,06,15,6.0,9.0,05,16,5


In [362]:
df13.shape

(510778, 30)

In [363]:
df13.to_csv('MoveInSyncData-final.csv', encoding='utf-8', index=False)